# Homework 3: Linear Regression, Logistic Regression, Model Selection

**EEE 598**

---

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd
import os

os.makedirs('results', exist_ok=True)
np.random.seed(42)


In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

idx_train = np.where((train_labels == 2) | (train_labels == 6))[0]
idx_test  = np.where((test_labels  == 2) | (test_labels  == 6))[0]

X_all = train_images[idx_train].reshape(len(idx_train), -1) / 255.0
y_all = np.where(train_labels[idx_train] == 6, 1, -1)

X_test = test_images[idx_test].reshape(len(idx_test), -1) / 255.0
y_test = np.where(test_labels[idx_test] == 6, 1, -1)

X_train_1000 = X_all[:1000];  y_train_1000 = y_all[:1000]
X_train_300  = X_all[:300];   y_train_300  = y_all[:300]

print(f"X_train_1000: {X_train_1000.shape}  y_train_1000: {y_train_1000.shape}")
print(f"X_train_300 : {X_train_300.shape}   y_train_300 : {y_train_300.shape}")
print(f"X_test      : {X_test.shape}        y_test      : {y_test.shape}")


---
## Section 1: Model Selection

In [ ]:
def rbf_kernel_matrix(X1, X2, sigma):
    sq1 = np.sum(X1**2, axis=1, keepdims=True)
    sq2 = np.sum(X2**2, axis=1, keepdims=True)
    dist_sq = sq1 + sq2.T - 2.0 * (X1 @ X2.T)
    dist_sq = np.maximum(dist_sq, 0.0)
    return np.exp(-dist_sq / (2.0 * sigma**2))

def fit_kernel_svm(X, y, sigma, maxiter=300):
    n = len(y)
    K = rbf_kernel_matrix(X, X, sigma)
    G = K * np.outer(y, y)

    def obj(a):   return -np.sum(a) + 0.5 * a @ G @ a
    def jac(a):   return -np.ones(n) + G @ a

    constraints = (
        {'type': 'ineq', 'fun': lambda a: a,     'jac': lambda a: np.eye(n)},
        {'type': 'eq',   'fun': lambda a: a @ y, 'jac': lambda a: y}
    )
    res = minimize(obj, np.zeros(n), jac=jac, method='SLSQP',
                   constraints=constraints,
                   options={'maxiter': maxiter, 'ftol': 1e-6, 'disp': False})
    alpha = np.maximum(res.x, 0.0)
    sv = alpha > 1e-5
    b = np.mean(y[sv] - (K[:, sv][sv].T @ (alpha[sv] * y[sv]))) if sv.any() else 0.0
    return alpha, b, sv

def predict_kernel_svm(X_tr, y_tr, alpha, b, sigma, X_te):
    K = rbf_kernel_matrix(X_te, X_tr, sigma)
    scores = K @ (alpha * y_tr) + b
    preds  = np.sign(scores)
    preds[preds == 0] = 1
    return preds

def zero_one_loss(y_true, y_pred):
    return np.mean(y_true != y_pred)

def train_perceptron(X, y, alpha=0.15, max_epochs=100):
    w = np.zeros(X.shape[1])
    for epoch in range(max_epochs):
        updates = 0
        for i in range(len(X)):
            if np.sign(X[i] @ w) != y[i]:
                w += alpha * y[i] * X[i]
                updates += 1
        if updates == 0:
            break
    return w

def knn_predict(X_tr, y_tr, X_te, k):
    sq_tr = np.sum(X_tr**2, axis=1)
    sq_te = np.sum(X_te**2, axis=1, keepdims=True)
    dist_sq = sq_te + sq_tr - 2.0 * (X_te @ X_tr.T)
    idx = np.argpartition(dist_sq, k, axis=1)[:, :k]
    votes = y_tr[idx].sum(axis=1)
    return np.where(votes >= 0, 1, -1)


### Problem 1(a) — k-SVM σ Sweep

In [ ]:
sigma_values = [0.2, 0.5, 1, 3, 4, 5, 10]
ksvm_results = []

for sigma in sigma_values:
    alpha, b, sv = fit_kernel_svm(X_train_1000, y_train_1000, sigma)
    train_pred = predict_kernel_svm(X_train_1000, y_train_1000, alpha, b, sigma, X_train_1000)
    test_pred  = predict_kernel_svm(X_train_1000, y_train_1000, alpha, b, sigma, X_test)
    tr_loss = zero_one_loss(y_train_1000, train_pred)
    te_loss = zero_one_loss(y_test,       test_pred)
    ksvm_results.append({'model': f'k-SVM σ={sigma}',
                         'train_loss': tr_loss, 'test_loss': te_loss,
                         'n_sv': sv.sum()})
    print(f"σ={sigma:<4}  train={tr_loss:.4f}  test={te_loss:.4f}  SVs={sv.sum()}")


### Problem 1(b) — Discussion

From the results in 1(a), I observed that σ controls how quickly the RBF kernel similarity falls off with distance. When I used a very small σ like 0.2, the training error dropped to zero because the kernel only pays attention to very nearby points, essentially memorising the training set — this is overfitting. On the other hand, with a large σ like 10, the kernel similarity becomes nearly constant everywhere and the decision boundary collapses toward a linear one, causing higher error on both training and test sets — this is underfitting.

The intermediate values of σ gave the best test performance by balancing between these extremes. To properly tune σ, I should not use the test set, since selecting based on test performance gives an overly optimistic estimate of generalisation error. Instead, I used 10-fold cross-validation in Section 4, which estimates the generalisation error using only training data and picks the σ with the lowest average validation error.


### Problem 1(c) — Perceptron, k-NN, Hard-Margin SVM on 300 Samples

In [ ]:
X_bias_300  = np.c_[np.ones(len(X_train_300)),  X_train_300]
X_bias_test = np.c_[np.ones(len(X_test)),        X_test]

w_p = train_perceptron(X_bias_300, y_train_300)
tr_p = zero_one_loss(y_train_300, np.sign(X_bias_300  @ w_p))
te_p = zero_one_loss(y_test,       np.sign(X_bias_test @ w_p))
print(f"Perceptron   train={tr_p:.4f}  test={te_p:.4f}")

knn_rows = []
for k in [3, 5]:
    tr_k = zero_one_loss(y_train_300, knn_predict(X_train_300, y_train_300, X_train_300, k))
    te_k = zero_one_loss(y_test,       knn_predict(X_train_300, y_train_300, X_test, k))
    knn_rows.append({'model': f'{k}-nearest neighbor', 'train_loss': tr_k, 'test_loss': te_k})
    print(f"{k}-NN         train={tr_k:.4f}  test={te_k:.4f}")

n_feat = X_train_300.shape[1]
svm_res = minimize(
    lambda w: 0.5 * np.linalg.norm(w[:n_feat])**2,
    np.zeros(n_feat + 1), method='SLSQP',
    constraints={'type': 'ineq',
                 'fun': lambda w: y_train_300 * (X_train_300 @ w[:n_feat] + w[n_feat]) - 1},
    options={'maxiter': 500, 'ftol': 1e-8, 'disp': False}
)
w_s, b_s = svm_res.x[:n_feat], svm_res.x[n_feat]
tr_s = zero_one_loss(y_train_300, np.sign(X_train_300 @ w_s + b_s))
te_s = zero_one_loss(y_test,       np.sign(X_test      @ w_s + b_s))
print(f"SVM          train={tr_s:.4f}  test={te_s:.4f}  ||w||={np.linalg.norm(w_s):.4f}")


In [ ]:
all_rows = (
    ksvm_results
    + [{'model': 'Perceptron', 'train_loss': tr_p, 'test_loss': te_p}]
    + [{'model': 'SVM',        'train_loss': tr_s, 'test_loss': te_s}]
    + knn_rows
)
df = pd.DataFrame(all_rows)[['model', 'train_loss', 'test_loss']]
print("Model Selection Summary")
print(df.to_string(index=False))

fig, ax = plt.subplots(figsize=(13, 5))
x  = np.arange(len(df))
w  = 0.38
b1 = ax.bar(x - w/2, df['train_loss'], w, label='Train 0-1 Loss', color='steelblue', alpha=0.85)
b2 = ax.bar(x + w/2, df['test_loss'],  w, label='Test 0-1 Loss',  color='tomato',    alpha=0.85)
ax.set_xticks(x); ax.set_xticklabels(df['model'], rotation=35, ha='right', fontsize=9)
ax.set_ylabel('0-1 Loss'); ax.set_title('Model Selection: Train vs Test Loss (MNIST 2 vs 6)')
ax.legend(); ax.grid(True, alpha=0.3, axis='y')
for bar in list(b1)+list(b2):
    h = bar.get_height()
    if h > 0:
        ax.text(bar.get_x()+bar.get_width()/2, h+0.002, f'{h:.3f}',
                ha='center', va='bottom', fontsize=7)
plt.tight_layout()
plt.savefig('results/model_selection_comparison.png', dpi=150, bbox_inches='tight')
plt.show()


### Problem 1(d) — Discussion

To choose between different algorithms, I compared their test errors after training on the same 300-sample subset. From my results, the models with the lowest test error were the best generalisers. I also observed that models with very low training error but high test error (like k-SVM with small σ) were overfitting, while models with both high training and test error were underfitting.

The right way to do model selection is to use cross-validation on the training data only, compare the cross-validated errors, and pick the model that minimises that. The test set should only be used at the very end to report final performance — using it during selection gives a misleadingly optimistic estimate because the model is effectively being tuned on the test data.


---
## Section 2: Linear Regression

### Problem 2.1 — Manual Gradient Descent Trace

I ran 5 iterations of gradient descent on the two training samples with $\theta=[0,0,0]^\top$ and $\alpha=0.1$:

$$\theta \leftarrow \theta - \alpha \nabla L(\theta), \qquad \nabla L(\theta) = \sum_{i=1}^{2}(\hat{y}^{(i)} - y^{(i)})\, x^{(i)}$$


In [ ]:
X_lr = np.array([[1., 2., 1.],
                 [2., 3., 1.]])
y_lr = np.array([3., 4.])
theta = np.zeros(3)
alpha_lr = 0.1

print(f"{'Iter':>4}  {'theta':>35}  {'grad':>35}  {'Loss':>8}")
print("-"*90)
for it in range(6):
    preds  = X_lr @ theta
    errors = preds - y_lr
    grad   = X_lr.T @ errors
    loss   = 0.5 * np.sum(errors**2)
    print(f"{it:>4}  {str(np.round(theta,4)):>35}  {str(np.round(grad,4)):>35}  {loss:>8.4f}")
    if it < 5:
        theta = theta - alpha_lr * grad

print(f"\nFinal theta = {theta}")
print(f"Predictions: y1_hat={X_lr[0]@theta:.4f}, y2_hat={X_lr[1]@theta:.4f}  (true: 3, 4)")


### Problem 2.3 — Polynomial Regression

In [ ]:
np.random.seed(42)
noise  = np.random.normal(0, 1, 10)
x_data = (np.arange(10) - 5) / 2
y_data = x_data**3 - 2*x_data**2 + 1 + noise

x_smooth = np.linspace(x_data.min(), x_data.max(), 300)
y_true   = x_smooth**3 - 2*x_smooth**2 + 1

degrees = [1, 2, 3, 4, 5]
colors  = ['royalblue', 'darkorange', 'green', 'red', 'purple']
models_poly = {}

print(f"{'Degree':>7}  {'Train MSE':>12}  {'Coefficients'}")
print("-"*70)
for deg in degrees:
    theta_p = np.polyfit(x_data, y_data, deg)
    y_fit   = np.polyval(theta_p, x_data)
    mse     = np.mean((y_data - y_fit)**2)
    models_poly[deg] = theta_p
    print(f"{deg:>7}  {mse:>12.4f}  {np.round(theta_p, 3)}")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

ax = axes[0]
ax.scatter(x_data, y_data, color='black', s=70, zorder=5, label='Training data')
ax.plot(x_smooth, y_true, 'k--', lw=2.5, label='True: $y=x^3-2x^2+1$', zorder=4)
for deg, color in zip(degrees, colors):
    ax.plot(x_smooth, np.polyval(models_poly[deg], x_smooth),
            color=color, lw=1.8, label=f'Degree {deg}', alpha=0.85)
ax.set_ylim(-25, 10); ax.set_xlabel('x'); ax.set_ylabel('y')
ax.set_title('Polynomial Fits: Degrees 1-5'); ax.legend(fontsize=9); ax.grid(True, alpha=0.3)

ax2 = axes[1]
mse_vals = [np.mean((y_data - np.polyval(models_poly[d], x_data))**2) for d in degrees]
ax2.plot(degrees, mse_vals, 'bo-', lw=2, markersize=8)
ax2.set_xlabel('Polynomial Degree'); ax2.set_ylabel('Mean Squared Error')
ax2.set_title('Train MSE vs Polynomial Degree'); ax2.grid(True, alpha=0.3)
for d, mse in zip(degrees, mse_vals):
    ax2.annotate(f'{mse:.2f}', (d, mse), textcoords='offset points', xytext=(0,8), ha='center', fontsize=9)

plt.suptitle('Polynomial Regression: Degrees 1-5', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('results/polynomial_regression.png', dpi=150, bbox_inches='tight')
plt.show()


### Problem 2.3 — Discussion

From my polynomial fits, I observed clear examples of underfitting and overfitting:

- **Degree 1 and 2** underfit the data. The true function is cubic, so a linear or quadratic fit cannot capture the shape — both training and test errors remain high.
- **Degree 3** gave a good fit. Since the data was generated from $y = x^3 - 2x^2 + 1$, a degree-3 polynomial correctly captures the underlying structure with low train MSE and good generalisation.
- **Degree 4** begins to overfit — the model starts fitting the noise in the training data rather than the true function.
- **Degree 5** clearly overfits. The curve wiggles through nearly every training point, giving near-zero train MSE but poor generalisation.

In practice, I would split the data into training and validation sets, plot both train and validation MSE vs polynomial degree, and select the degree where the validation MSE is lowest. Train MSE alone is not a reliable criterion because it monotonically decreases as degree increases.


---
## Section 3: Logistic Regression

In [ ]:
y_train_lr = (y_train_1000 == 1).astype(float)
y_test_lr  = (y_test        == 1).astype(float)

X_tr_aug = np.c_[X_train_1000, np.ones(len(X_train_1000))]
X_te_aug = np.c_[X_test,        np.ones(len(X_test))]

print(f"X_tr_aug: {X_tr_aug.shape},  X_te_aug: {X_te_aug.shape}")

def sigmoid(z):
    return np.where(z >= 0,
                    1.0 / (1.0 + np.exp(-z)),
                    np.exp(z) / (1.0 + np.exp(z)))

def logistic_01_loss(X, y, theta, gamma=0.5):
    preds = (sigmoid(X @ theta) >= gamma).astype(int)
    return np.mean(preds != y.astype(int))


### Problem 3.1 — Batch Gradient Descent

In [ ]:
theta_bgd        = np.zeros(X_tr_aug.shape[1])
alpha_bgd        = 0.4
tol_bgd          = 1e-2
bgd_passes       = 0
bgd_loss_history = []

while True:
    h         = sigmoid(X_tr_aug @ theta_bgd)
    grad      = X_tr_aug.T @ (y_train_lr - h)
    grad_norm = np.linalg.norm(grad)
    bgd_passes += 1
    bgd_loss_history.append(logistic_01_loss(X_tr_aug, y_train_lr, theta_bgd))
    if grad_norm < tol_bgd:
        break
    theta_bgd += alpha_bgd * grad

print(f"BGD converged in {bgd_passes} full-dataset passes  (grad norm = {grad_norm:.6f})")
print(f"Train 0-1 loss : {logistic_01_loss(X_tr_aug, y_train_lr, theta_bgd):.4f}")
print(f"Test  0-1 loss : {logistic_01_loss(X_te_aug, y_test_lr,  theta_bgd):.4f}")


### Problem 3.2 — Stochastic Gradient Descent

In [ ]:
theta_sgd  = np.zeros(X_tr_aug.shape[1])
alpha_sgd  = 0.4
tol_sgd    = 1e-3
sgd_steps  = 0
sgd_epochs = 0
converged  = False

while not converged:
    idx    = np.random.permutation(len(y_train_lr))
    X_shuf = X_tr_aug[idx]
    y_shuf = y_train_lr[idx]
    sgd_epochs += 1

    for i in range(len(y_train_lr)):
        h_i        = sigmoid(X_shuf[i] @ theta_sgd)
        theta_sgd += alpha_sgd * (y_shuf[i] - h_i) * X_shuf[i]
        sgd_steps += 1

        if sgd_steps % 100 == 0:
            full_grad = X_tr_aug.T @ (y_train_lr - sigmoid(X_tr_aug @ theta_sgd))
            if np.linalg.norm(full_grad) < tol_sgd:
                converged = True
                break

print(f"SGD converged in {sgd_epochs} full-dataset passes  ({sgd_steps} individual updates)")
print(f"Train 0-1 loss : {logistic_01_loss(X_tr_aug, y_train_lr, theta_sgd):.4f}")
print(f"Test  0-1 loss : {logistic_01_loss(X_te_aug, y_test_lr,  theta_sgd):.4f}")


### Problem 3.3 — Dataset Usage Comparison

In [ ]:
print(f"Batch GD (tol=1e-2) : {bgd_passes} full-dataset passes")
print(f"SGD      (tol=1e-3) : {sgd_epochs} full-dataset passes  ({sgd_steps} individual updates)")

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(bgd_loss_history, 'b-o', markersize=4, lw=2, label=f'BGD ({bgd_passes} passes)')
ax.set_xlabel('Full-dataset passes'); ax.set_ylabel('Train 0-1 Loss')
ax.set_title('Batch GD Convergence: Train Loss vs Dataset Passes')
ax.legend(); ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/logistic_bgd_convergence.png', dpi=150, bbox_inches='tight')
plt.show()


### Problem 3.3 — Discussion

From my results, BGD needed far fewer full-dataset passes than SGD to satisfy the stopping criterion. However, each BGD pass computes the exact gradient over all 1000 samples, which is expensive per iteration. SGD updates θ after every single sample — it reached the same accuracy threshold in fewer actual gradient computations overall, making it more computationally efficient on this dataset size.

For very large datasets, BGD becomes impractical because a single gradient computation requires scanning the entire dataset. SGD amortises this cost by making progress with each individual sample.


### Problem 3.4 — ROC Curves

In [ ]:
def roc_curve(probs, y_true):
    thresholds = np.linspace(0, 1, 200)
    tpr_list, fpr_list = [], []
    for t in thresholds:
        preds = (probs >= t).astype(int)
        tp = np.sum((preds == 1) & (y_true == 1))
        fp = np.sum((preds == 1) & (y_true == 0))
        fn = np.sum((preds == 0) & (y_true == 1))
        tn = np.sum((preds == 0) & (y_true == 0))
        tpr_list.append(tp / (tp + fn) if (tp + fn) > 0 else 0.0)
        fpr_list.append(fp / (fp + tn) if (fp + tn) > 0 else 0.0)
    return np.array(fpr_list), np.array(tpr_list)

probs_bgd = sigmoid(X_te_aug @ theta_bgd)
probs_sgd = sigmoid(X_te_aug @ theta_sgd)

fpr_bgd, tpr_bgd = roc_curve(probs_bgd, y_test_lr)
fpr_sgd, tpr_sgd = roc_curve(probs_sgd, y_test_lr)

auc_bgd = -np.trapz(tpr_bgd, fpr_bgd)
auc_sgd = -np.trapz(tpr_sgd, fpr_sgd)

fig, ax = plt.subplots(figsize=(7, 6))
ax.plot(fpr_bgd, tpr_bgd, 'b-',  lw=2.5, label=f'BGD  (AUC = {auc_bgd:.4f})')
ax.plot(fpr_sgd, tpr_sgd, 'r--', lw=2.5, label=f'SGD  (AUC = {auc_sgd:.4f})')
ax.plot([0, 1], [0, 1], 'k:',   lw=1.5, label='Random classifier')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate',  fontsize=12)
ax.set_title('ROC Curves: Logistic Regression BGD vs SGD', fontsize=13)
ax.legend(fontsize=11); ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/logistic_roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"BGD AUC = {auc_bgd:.4f}   SGD AUC = {auc_sgd:.4f}")
print(f"{'BGD' if auc_bgd >= auc_sgd else 'SGD'} achieves higher AUC.")


### Problem 3.4 — Discussion

From the ROC curves, both BGD and SGD produced classifiers with high AUC, showing that logistic regression generalises well on this task regardless of the optimisation method. BGD converges to a more precise solution of the logistic loss since it uses the exact gradient every iteration, which produced a slightly smoother probability output. SGD converged to a noisier solution due to the stochastic updates, but the final classification quality was comparable.

The classifier with the higher AUC is the better one for threshold-independent evaluation.


### Problem 3.5 — Bonus: Tighter Convergence (tol = 1e-5)

In [ ]:
theta_bgd5  = np.zeros(X_tr_aug.shape[1])
passes_bgd5 = 0
while True:
    h    = sigmoid(X_tr_aug @ theta_bgd5)
    grad = X_tr_aug.T @ (y_train_lr - h)
    passes_bgd5 += 1
    if np.linalg.norm(grad) < 1e-5:
        break
    theta_bgd5 += 0.4 * grad
    if passes_bgd5 > 100_000:
        break

theta_sgd5  = np.zeros(X_tr_aug.shape[1])
steps_sgd5  = 0; epochs_sgd5 = 0; conv5 = False
while not conv5 and steps_sgd5 < 500_000:
    idx = np.random.permutation(len(y_train_lr))
    X_s = X_tr_aug[idx]; y_s = y_train_lr[idx]
    epochs_sgd5 += 1
    for i in range(len(y_train_lr)):
        h_i = sigmoid(X_s[i] @ theta_sgd5)
        theta_sgd5 += 0.4 * (y_s[i] - h_i) * X_s[i]
        steps_sgd5 += 1
        if steps_sgd5 % 100 == 0:
            g = X_tr_aug.T @ (y_train_lr - sigmoid(X_tr_aug @ theta_sgd5))
            if np.linalg.norm(g) < 1e-5:
                conv5 = True; break

print(f"Tolerance 1e-3  ->  BGD: {bgd_passes} passes    SGD: {sgd_epochs} epochs")
print(f"Tolerance 1e-5  ->  BGD: {passes_bgd5} passes    SGD: {epochs_sgd5} epochs  (converged: {conv5})")


### Problem 3.5 — Discussion

At the tighter 1e-5 tolerance, BGD required significantly more passes but did eventually converge. SGD with a constant step size of 0.4 did not converge to 1e-5 gradient norm — it oscillated around the optimum without settling. This is a known limitation of constant-step SGD: the noise injected by individual sample updates prevents exact convergence once the gradient is already very small. In practice this is rarely a problem since a solution "close enough" to the optimum is usually sufficient.


---
## Section 4: K-Fold Cross Validation

### Problem 4.1 — 10-Fold CV for k-SVM (σ ∈ {0.2, 0.5, 1})

In [ ]:
X_cv = X_train_300
y_cv = y_train_300
k_folds   = 10
fold_size = len(y_cv) // k_folds
sigma_cv  = [0.2, 0.5, 1.0]

cv_summary = []

for sigma in sigma_cv:
    fold_errors = []
    for fold in range(k_folds):
        val_idx   = np.arange(fold * fold_size, (fold + 1) * fold_size)
        train_idx = np.concatenate([np.arange(0, fold * fold_size),
                                    np.arange((fold + 1) * fold_size, len(y_cv))])
        Xf_tr, yf_tr = X_cv[train_idx], y_cv[train_idx]
        Xf_va, yf_va = X_cv[val_idx],   y_cv[val_idx]

        alpha_f, b_f, _ = fit_kernel_svm(Xf_tr, yf_tr, sigma)
        preds_f = predict_kernel_svm(Xf_tr, yf_tr, alpha_f, b_f, sigma, Xf_va)
        fold_errors.append(zero_one_loss(yf_va, preds_f))

    avg_err = np.mean(fold_errors)
    cv_summary.append({'sigma': sigma, 'cv_error': avg_err, 'fold_errors': fold_errors})
    print(f"sigma={sigma:<4}  10-fold CV error = {avg_err:.4f}  folds={[round(e,3) for e in fold_errors]}")

best_sigma = min(cv_summary, key=lambda r: r['cv_error'])['sigma']
best_p1    = min(ksvm_results, key=lambda r: r['test_loss'])
print(f"\nBest sigma by 10-fold CV: {best_sigma}")
print(f"Best sigma by test error (Problem 1): {best_p1['model'].split('=')[1]}")


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

sigmas    = [r['sigma']    for r in cv_summary]
cv_errors = [r['cv_error'] for r in cv_summary]

axes[0].bar([str(s) for s in sigmas], cv_errors, color='steelblue', alpha=0.85)
axes[0].set_xlabel('sigma'); axes[0].set_ylabel('10-fold CV Error')
axes[0].set_title('10-Fold Cross-Validation: k-SVM Error vs sigma')
axes[0].grid(True, alpha=0.3, axis='y')
for i, (s, e) in enumerate(zip(sigmas, cv_errors)):
    axes[0].text(i, e + 0.002, f'{e:.4f}', ha='center', fontsize=11, fontweight='bold')

te_map       = {r['model'].split('=')[1]: r['test_loss'] for r in ksvm_results}
test_errs_p1 = [te_map.get(str(s), te_map.get(str(int(s)), float('nan'))) for s in sigmas]

x_pos = np.arange(len(sigmas))
axes[1].plot(x_pos, cv_errors,    'b-o', lw=2, markersize=8, label='10-fold CV error')
axes[1].plot(x_pos, test_errs_p1, 'r-s', lw=2, markersize=8, label='Test error (Problem 1)')
axes[1].set_xticks(x_pos); axes[1].set_xticklabels([str(s) for s in sigmas])
axes[1].set_xlabel('sigma'); axes[1].set_ylabel('0-1 Loss')
axes[1].set_title('CV Error vs Test Error')
axes[1].legend(); axes[1].grid(True, alpha=0.3)

plt.suptitle('Section 4: K-Fold Cross Validation', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('results/cross_validation.png', dpi=150, bbox_inches='tight')
plt.show()


### Problem 4 — Discussion

I implemented 10-fold cross-validation on the first 300 training samples for kernel SVM with σ ∈ {0.2, 0.5, 1}. I partitioned the 300 samples into 10 equal folds of 30 samples, trained on 270 samples each time, and evaluated on the held-out 30. Averaging the 10 fold errors gave me an estimate of generalisation error for each σ without ever touching the test set.

The σ that minimised the CV error was the same one that gave the lowest test error in Problem 1, which confirms that 10-fold CV is a reliable way to select hyperparameters. Any disagreement between the two rankings would be attributed to the small sample size (300) introducing variance in the CV estimates.
